# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770499268004                   -0.53    9.0         
  2   -2.771694203313       -2.92       -1.32    1.0    134ms
  3   -2.771714489303       -4.69       -2.50    1.0    164ms
  4   -2.771714656363       -6.78       -3.25    1.0    197ms
  5   -2.771714712077       -7.25       -3.80    2.0    162ms
  6   -2.771714715110       -8.52       -4.66    1.0    149ms
  7   -2.771714715241       -9.88       -5.04    2.0    188ms
  8   -2.771714715248      -11.13       -5.59    1.0    149ms
  9   -2.771714715250      -11.93       -6.75    2.0    187ms
 10   -2.771714715250   +  -14.10       -6.97    1.0    154ms
 11   -2.771714715250      -14.21       -8.28    1.0    169ms


-0.00013457252411415628

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770367212668                   -0.53    8.0         
  2   -2.771765272693       -2.85       -1.30    1.0    179ms
  3   -2.771801103834       -4.45       -2.61    1.0    185ms
  4   -2.771802068605       -6.02       -3.78    2.0    216ms
  5   -2.771802073882       -8.28       -4.10    2.0    163ms
  6   -2.771802074448       -9.25       -5.02    1.0    185ms
  7   -2.771802074475      -10.57       -5.45    2.0    169ms
  8   -2.771802074476      -11.95       -6.01    1.0    158ms
  9   -2.771802074476      -12.83       -7.20    2.0    182ms
 10   -2.771802074476      -13.61       -7.33    2.0    190ms
 11   -2.771802074476      -14.65       -8.77    1.0    154ms


0.017612221163054914

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457252411415628
Displaced dipole:  0.017612221163054914
Polarizability :   1.7746793687169071


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920640706e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766550897662e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852759235557e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694593845623e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787730608e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.275609836066e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 7.408170910708e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 7.408170910708e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.768942296681e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 3.536954925527e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 5.007307842761e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 6.404153371468e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 6.404153371468e-14
┌ Info: GMRES linsolve converged at iteration 2, step

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.